In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
data_pop_densls=gpd.read_file('sls_builtup_5300.gpkg')
data_degurba=gpd.read_file('degurba_5300_rev_w_buitup_adjustment_3.gpkg')

In [3]:
data_pop_dens_degurba=gpd.overlay(data_degurba,data_pop_densls,how='intersection')

C:\Users\user\AppData\Local\Temp\ipykernel_9560\3972822690.py:1: UserWarning: `keep_geom_type=True` in overlay resulted in 5 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  data_pop_dens_degurba=gpd.overlay(data_degurba,data_pop_densls,how='intersection')


In [4]:
data_pop_dens_degurba.head(2)

,Name,pop_built_s2,is_U_CENTRE,per_built_after,is_U_CLUSTER,classification,idsls,TOTAL_PEND,area_pop,area_built,area_built_total,per_built,pop_built,geometry
0,3191526924199395328,159.0,0,2.0,0,RURAL,53170110090008,80.0,617668.249234,11443.556546,11443.556546,100.0,80.0,"POLYGON Z ((119.04335 -9.66882 0.00000, 119.04..."
1,3191526924199395328,159.0,0,2.0,0,RURAL,53170110090009,79.0,141235.576827,4155.479646,4155.479646,100.0,79.0,"POLYGON Z ((119.04332 -9.66941 0.00000, 119.04..."


In [5]:
data_pop_dens_degurba['iddesa']=data_pop_dens_degurba.idsls.apply(lambda y: str(y)[:10])

In [6]:
tab_classi=pd.pivot_table(data_pop_dens_degurba,index='iddesa',columns='classification',
                          values='pop_built',aggfunc='sum').fillna(0).reset_index()

In [7]:
def define_LAU(y):
    total_pop=y.CENTRE+y.CLUSTER+y.RURAL
    if total_pop==0:
        return 'RURALS'
    elif (y.CENTRE/total_pop)>=0.5:
        return 'CITIES'
    elif (((y.CENTRE/total_pop)<0.5)&((y.RURAL/total_pop)<0.5)):
        return 'TOWNS'
    else:
        return 'RURALS'

In [8]:
tab_classi['LAU']=tab_classi.apply(lambda y:define_LAU(y),axis=1)

In [9]:
peta_desa=gpd.read_file('DESA_20201_BYINDO.gpkg')

In [10]:
from dbfread import DBF

In [11]:
mfd_2020_1=pd.DataFrame(iter(DBF('mfd_2020_1.dbf')))

In [12]:
peta_desa_5100=peta_desa.query('kdprov==\'53\'')

In [13]:
peta_desa_5100=peta_desa_5100.merge(tab_classi[['iddesa','LAU']],how='left').fillna('RURALS')

In [14]:
mfd_desa_5100_UR=mfd_2020_1.query('PROP==\'53\'').assign(iddesa=lambda y:y.PROP+y.KAB+y.KEC+y.DESA)[['iddesa','DRH']]

In [15]:
peta_desa_5100.merge(mfd_desa_5100_UR,how='left').rename(
    columns={'DRH':'UR'}).fillna({'UR':2}).to_file('desa_5300_LAU_UR_rev2.gpkg',driver='GPKG')

In [16]:
peta_desa_5100.merge(mfd_desa_5100_UR,how='left').rename(
    columns={'DRH':'UR'}).fillna({'UR':2}).pivot_table(columns='LAU',index='UR',values='iddesa',aggfunc='count')

LAU,CITIES,RURALS,TOWNS
UR,,,
1,107,8,84
2,35,2190,929


In [17]:
peta_desa_5100.merge(mfd_desa_5100_UR,how='left').rename(
    columns={'DRH':'UR'}).fillna({'UR':2}).query('(UR=="1") and (LAU=="RURALS")')

,iddesa,idkab,idkec,nmprov,nmkab,nmkec,nmdesa,kdprov,kdkab,kdkec,kddesa,sumber,periode,geometry,LAU,UR
265,5303111010,5303,5303111,NUSA TENGGARA TIMUR,KUPANG,NEKAMESE,OBEN,53,03,111,010,BPS,2020_1,"POLYGON ((123.72423 -10.23693, 123.72493 -10.2...",RURALS,1
969,5307010046,5307,5307010,NUSA TENGGARA TIMUR,ALOR,PANTAR,KABIR,53,07,010,046,BPS,2020_1,"POLYGON ((124.21920 -8.25843, 124.21902 -8.258...",RURALS,1
1759,5311020019,5311,5311020,NUSA TENGGARA TIMUR,ENDE,ENDE,NDETUNDORA III,53,11,020,019,BPS,2020_1,"POLYGON ((121.65034 -8.79110, 121.64954 -8.791...",RURALS,1
1782,5311031002,5311,5311031,NUSA TENGGARA TIMUR,ENDE,ENDE TIMUR,KEL. REWARANGGA,53,11,031,002,BPS,2020_1,"POLYGON ((121.66883 -8.78176, 121.66900 -8.781...",RURALS,1
1783,5311031003,5311,5311031,NUSA TENGGARA TIMUR,ENDE,ENDE TIMUR,KEDEBODU,53,11,031,003,BPS,2020_1,"POLYGON ((121.65985 -8.79541, 121.66112 -8.796...",RURALS,1
1803,5311040003,5311,5311040,NUSA TENGGARA TIMUR,ENDE,NDONA,MANULONDO,53,11,040,003,BPS,2020_1,"POLYGON ((121.70871 -8.82716, 121.70901 -8.827...",RURALS,1
2753,5317060003,5317,5317060,NUSA TENGGARA TIMUR,SUMBA BARAT DAYA,WEWEWA TIMUR,PADA EWATA,53,17,060,003,BPS,2020_1,"POLYGON ((119.31893 -9.60128, 119.31913 -9.601...",RURALS,1
2761,5317060021,5317,5317060,NUSA TENGGARA TIMUR,SUMBA BARAT DAYA,WEWEWA TIMUR,KADI WONE,53,17,060,021,BPS,2020_1,"POLYGON ((119.32498 -9.60713, 119.32507 -9.607...",RURALS,1
